# L10 - Scientific Programming with SciPy

The [SciPy library](https://docs.scipy.org/doc/scipy/reference/) is part of the [SciPy Software Stack](https://scipy.org/index.html), which includes many of the packages that we have already worked with, such as `numpy`, `pandas` and `matplotlib`. It contains lots of powerful tools for various tasks in mathematics, statistics and signal processing. Generally, if a certain method or algorithm has experienced widespread use in the computational sciences, there is a good chance that it is provided somewhere in `scipy`.

In [ ]:
!pip install scipy
import scipy

Here are some of the submodules of the `scipy` library which we will take a closer look at:

* [`scipy.linalg`](https://docs.scipy.org/doc/scipy/reference/linalg.html): Linear Algebra
* [`scipy.interpolate`](https://docs.scipy.org/doc/scipy/reference/interpolate.html): Interpolation
* [`scipy.integrate`](https://docs.scipy.org/doc/scipy/reference/integrate.html): Integration and ODEs
* [`scipy.optimize`](https://docs.scipy.org/doc/scipy/reference/optimize.html): (Non-Linear) Optimization
* [`scipy.spatial`](https://docs.scipy.org/doc/scipy/reference/spatial.html): Spatial Algorithms
* [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html): Statistics and Distributions

Here are some honourable mentions of `scipy` submodules that we will not cover in this lecture:

* [`scipy.special`](https://docs.scipy.org/doc/scipy/reference/special.html): Special Functions (such as `softmax`)
* [`scipy.fft`](https://docs.scipy.org/doc/scipy/reference/fft.html): Discrete Fourier Transforms
* [`scipy.io`](https://docs.scipy.org/doc/scipy/reference/io.html): Utilities for File Handling
* [`scipy.cluster`](https://docs.scipy.org/doc/scipy/reference/cluster.html): Clustering Algorithms

## Linear Algebra

The `linalg` submodule includes many useful functions for doing linear algebra. For example, it contains functions for computing a determinant, inverting matrices, computing eigenvalues or different decompositions. It also includes multiple solvers for linear equations systems.

In [ ]:
from scipy import linalg

### Matrix Inverse
Create a matrix of which we want to get the inverse.

In [ ]:
import numpy as np

mat = np.array([[ 0, -1, -2], [-1,  2,  3], [ 0,  1,  0]])
mat

Note: In this context, a *matrix* is just an `ndarray`, not the array-like object [`np.matrix`](https://numpy.org/doc/stable/reference/generated/numpy.matrix.html), which will be faded out soon. 

Compute the inverse matrix using `scipy.linalg.inv`.

In [ ]:
inv = linalg.inv(mat)
inv

The dot product of a matrix with its inverse should yield the identity matrix: $XX^{-1}=I$

In [ ]:
print("Inverse of inverse equals original matrix?", np.allclose(mat, linalg.inv(inv)))
np.dot(inv, mat)

We can only compute the inverse of matrices with a determinant unequal 0. Matrices with a determinant of 0 are also called singular matrices.

In [ ]:
sing = np.array([[ 1,  2,  3], [-1, -2, -3], [ 0,  1,  0]])
sing

In [ ]:
linalg.det(sing)

SciPy will check if the matrix is singular and throws an error if we try to invert it.

In [ ]:
linalg.inv(sing)

### Solving Linear Equations
The [`scipy.linalg.solve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve.html#scipy-linalg-solve) function can solve linear equation sets of the form $Ax=b$ fox $x$, given a square matrix $a$ and vector/matrix $b$.

$
A=\pmatrix{a_{11} & a_{12} \\ a_{21} & a_{22}}=\pmatrix{-2 & 6 \\ 4 & 2}\\
b=\pmatrix{b_1 \\ b_2}=\pmatrix{3 \\ -2}\\
x=\pmatrix{x_1 \\ x_2}
$

What are the values for $x$ such that $Ax=b$ is satisfied?

$
\pmatrix{-2x_1 & 6x_2 \\ 4x_1 & 2x_2}=\pmatrix{3 \\ -2}
$

Create NumPy arrays for $A$ and $b$.

In [ ]:
A = np.array([[-2, 6], [4, 2]])
A

In [ ]:
b = np.array([3, -2])
b

Solve the set of equations for x.

In [ ]:
x = linalg.solve(A, b)
x

Is the result of $Ax$ really $b$?

In [ ]:
A @ x

### Eigenvalue Decomposition
SciPy includes multiple [functions for computing the eigenvalues of a matrix](https://docs.scipy.org/doc/scipy/reference/linalg.html#eigenvalue-problems). Let's have a look at the `eig` function, which returns the eigenvalues and eigenvectors of a matrix.

The eigenvalues of a matrix $M$ are scalars by which an eigenvector $v$ of $M$ is scaled when $v$ is transformed by $M$. An eigenvector $v$ of $M$ is a vector for which the product of $M$ and $v$ is a scaled version of $v$ (no rotation).

Generally, this can be written as $$Mv=\lambda v$$ where $v$ is an eigenvector of $M$ and $\lambda$ is the corresponding eigenvalue. In case you have not come across eigenvectors and eigenvalues yet, [here](https://www.youtube.com/watch?v=PFDu9oVAE-g&vl=en) is a good explain video.

Create an example matrix $M$.

In [ ]:
M = np.array([[1, 0.75], [0, -0.5]])
M

Let's have a look at how $M$ transforms the vector $x$.

In [ ]:
x = np.array([1, 1])
x_transformed = M @ x

x, x_transformed

In [ ]:
print(f'Norm of x: {linalg.norm(x):.3f}, norm of the transformed x: {linalg.norm(x_transformed):.3f}')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def draw_vectors(vecs, labels=None, ax=None, **kwargs):
    if ax is None:
        _, ax = plt.subplots(figsize=(7, 7))
        
    arrows = []
    for i, vec in enumerate(vecs):
        color = f'C{i}'
        arrows.append(ax.arrow(0, 0, *vec, width=0.025, length_includes_head=True, ec=color, fc=color))
        ax.annotate(str(vec), xy=vec + 0.2 * vec / linalg.norm(vec))
    
    ax.set(xlim=(-3, 3), ylim=(-3, 3), **kwargs)
    
    if labels is not None:
        plt.legend(arrows, labels)
    
draw_vectors([x, x_transformed], ['original', 'transformed'])

In [ ]:
eigvals, eigvecs = linalg.eig(M)
eigvecs = eigvecs.T
eigvals, eigvecs

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(15, 7))
labels = [f'Eigenvalue: {eigval.real:.2f}' for eigval in eigvals]
draw_vectors(eigvecs, labels, title='Eigenvectors', ax=axes[0])
draw_vectors([M @ eigvec for eigvec in eigvecs], title='Transformed eigenvectors', ax=axes[1])
plt.show()

SciPy's `eig` function is able to compute left and right eigenvectors. The most typical eigenvectors are right eigenvectors which are of the form $Mv=\lambda v$, while left eigenvectors correspond to $vM=\lambda v$.

In [ ]:
eigvals, eigvecs = linalg.eig(M, left=False, right=True)
eigval, eigvec = eigvals[0].real, eigvecs.T[0]

print('First eigenvector:', eigvec)
print('First eigenvalue:', eigval)

print('Mv / lambda =', M @ eigvec / eigval)
print('vM / lambda =', eigvec @ M / eigval)

By default `left=False` and `right=True`, since right eigenvalues are more common than left eigenvalues.

In [ ]:
eigvals, eigvecs = linalg.eig(M, left=True, right=False)
eigval, eigvec = eigvals[0].real, eigvecs.T[0]

print('First eigenvector:', eigvec)
print('First eigenvalue:', eigval)

print('Mv / lambda =', M @ eigvec / eigval)
print('vM / lambda =', eigvec @ M / eigval)

We can also get both at the same time.

In [ ]:
linalg.eig(M, left=True, right=True)

## Interpolation

We already had a brush with a basic interpolation in 2021-homework04. Generally, to interpolate is to estimate or predict unknown data points in between known data points.

In [ ]:
from scipy import interpolate

`scipy` has support for both univariate and multivariate interpolation. The basic univariate interpolation method is [`scipy.interpolate.interp1d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html#scipy.interpolate.interp1d).

In [ ]:
x_true = np.arange(10)

func_true = lambda x: 1 - 0.9**x
y_true = func_true(x_true)

func_pred = interpolate.interp1d(x_true, y_true)

x_pred = np.linspace(0, 9, 91)
y_pred = func_pred(x_pred)

fig, ax = plt.subplots()

ax.scatter(x_true, y_true, label="True Values", c="g")
ax.plot(x_pred, y_pred, label="Predicted Values", c="b")

fig.legend(loc="lower right")

Interpolation can also be used to find order where there is none:

In [ ]:
x_rand = np.random.permutation(50)[:10]
y_rand = np.random.randint(0, 10, size=10)

func_pred = interpolate.interp1d(x_rand, y_rand, kind="cubic") # try different kinds: "linear", "cubic", "next"

x_pred = np.linspace(x_rand.min(), x_rand.max())
y_pred = func_pred(x_pred)

fig, ax = plt.subplots()

ax.scatter(x_rand, y_rand, label="Random Values", c="g")
ax.plot(x_pred, y_pred, label="Predicted Values", c="b")

fig.legend(loc="lower right")

There is also a corresponding 2d variant, [`scipy.interpolate.interp2d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html#scipy-interpolate-interp2d):

In [ ]:
x_rand = np.random.permutation(10)
y_rand = np.random.permutation(10)
z_rand = np.random.randint(0, 10, size=10)

func_pred = scipy.interpolate.interp2d(x_rand, y_rand, z_rand, kind="linear")

x_pred = np.arange(10)
y_pred = np.arange(10)

z_pred = func_pred(x_pred, y_pred)

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

ax.scatter(x_rand, y_rand, z_rand, c="r")
ax.plot_surface(x_pred, y_pred, z_pred, cmap="Blues")


## Integration and ODEs

`scipy` has a dedicated submodule for integration called [`integrate`](https://docs.scipy.org/doc/scipy/reference/integrate.html). It contains functions for integrating multiple times in one go and even supports integration and solvers for Ordinary Differential Equations (ODEs).

In [ ]:
from scipy import integrate

Here we will only have a look at the [`integrate.quad`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html#scipy-integrate-quad) function, which can be used to compute a definite integral of some function.

### Integration

In [ ]:
def draw_integral(x, y, limits, integral, err, **kwargs):
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(x, y)
    
    integral_mask = (x >= limits[0]) & (x < limits[1])
    ax.fill_between(x[integral_mask], y[integral_mask], alpha=0.2)
    
    ax.axhline(0, c='black', alpha=0.5)
    ax.axvline(limits[0], c='black')
    ax.axvline(limits[1], c='black')
    
    ax.set(title=f'Integral result: {integral:.3f} (error: {err:.3g})', **kwargs)
    plt.show()

In [ ]:
func = lambda x: 1

x = np.linspace(-10, 10, 1000)
y = np.array(list(map(func, x)))

limits = (-5, 5)
integral, err = integrate.quad(func, *limits)
draw_integral(x, y, limits, integral, err)

In [ ]:
func = lambda x: 0.2 * x**3 - x**2 + 1

x = np.linspace(-2, 6, 1000)
y = np.array(list(map(func, x)))

limits = (-0.919, 4.781)
integral, err = integrate.quad(func, *limits)
draw_integral(x, y, limits, integral, err)

The `quad` function is numerically estimating the integration and does not check if a function can mathematically be integrated. It is therefore able to integrate some non-integratable functions but results might not be 100% accurate.

In [ ]:
func = lambda x: float(x >= 0)

x = np.linspace(-2, 2, 1000)
y = np.array(list(map(func, x)))

limits = (-1, 1)
integral, err = integrate.quad(func, *limits)
draw_integral(x, y, limits, integral, err)

### Derivative Estimation

In [ ]:
from scipy import misc

`scipy` also has a function to estimate the derivative of any function at a certain location. The function is called [`scipy.misc.derivative`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.misc.derivative.html#scipy.misc.derivative).

In [ ]:
func = lambda x: x ** 2

xs = np.linspace(-2, 2, 1000)
ys = func(xs)

x = 1
y = func(x)
dx, dy = 1, misc.derivative(func, x)

plt.plot(xs, ys)
plt.arrow(x, y, dx / 2, dy / 2, width=0.025, fc='black')
plt.title(f'Derivative at x={x}: {dy:.3f}')
plt.show()

### ODE Solving

`scipy` also has the ability solve the initial value problem for a system of [ODEs](https://en.wikipedia.org/wiki/Ordinary_differential_equation).  

The canonical way to do that used to be [`scipy.integrate.odeint`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html), but this interface is slowly being faded out in favour of the newer [`scipy.integrate.solve_ivp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html#scipy.integrate.solve_ivp).

There will be a bonus homework assignment this week which will go into more detail regarding ODEs.

## Optimization

Optimization, most commonly in the form of minimization or maximization, is the adjustment or selection of parameters according to some criterion. The parameters are said to be optimized with respect to the criterion.

In [ ]:
from scipy import optimize

The `minimize_scalar` function takes a univariate function as an argument. `minimize_scalar` minimizes the input variable with respect to the function's output. An `OptimizeResult` is returned, which includes information about the optimization process, e.g. if it succeeded, what the minimal function value was and where this minimum was found.

In [ ]:
func = lambda x: x ** 2 + 1
optimize.minimize_scalar(func)

In [ ]:
func = lambda x: 0.5 * x ** 3 - x ** 2

xs = np.linspace(-1, 2, 1000)
ys = func(xs)

res = optimize.minimize_scalar(func)

print(res)

plt.plot(xs, ys)
plt.plot(res.x, res.fun, 'ro')
plt.show()

[`scipy.optimize.minimize`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) can also be used to minimize the value of a multivariate function.

In [ ]:
func = lambda var: 0.6 * var[0]**4 - 2 * var[0]**3 + 5 * var[1]**2

res = optimize.minimize(func, (1, 1)) # you need to give an initial guess regarding a minimum

vals = np.linspace(-2.5, 3.5, 100)
xs, ys = np.meshgrid(vals, vals)
zs = np.empty(xs.shape)

for i, j in np.ndindex(xs.shape):
    zs[i,j] = func((xs[i,j], ys[i,j]))

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')

ax.contour(xs, ys, zs, 25, cmap='gray')
ax.scatter(*res.x, res.fun, c='g', s=250, marker='o')

plt.show()

However, be aware that the found minimum is not necessarily a *global* one - for that, there are more powerful global optimization functions such as [`scipy.optimize.dual_annealing`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.dual_annealing.html).

Additionally to finding minimums, the `optimize` submodule can also find the roots of functions.

In [ ]:
func = lambda x: 0.5 * x ** 3 - x ** 2 + 0.25

xs = np.linspace(-1, 3, 1000)
ys = func(xs)

res = optimize.root_scalar(func, x0=1, x1=3)

plt.plot(xs, ys, label="Function")
plt.plot(res.root, func(res.root), 'ro', label="Root")
plt.axhline(0, c='black', alpha=0.3)

plt.legend()

plt.show()

## Spatial Algorithms

`scipy` has a wide variety of algorithms related to spatial concepts.

In [ ]:
from scipy import spatial

The most common usage of `scipy.spatial` might be the distance functions in [`scipy.spatial.distance`](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html). There is a wide range of vector distances which you might already know from other lectures. A vector in this context is just a one-dimensional `ndarray`.

In [ ]:
numeric_vec_a, numeric_vec_b = np.random.randint(0, 100, size=(2, 2))

print("Euclidean: {:.3f}".format(spatial.distance.euclidean(numeric_vec_a, numeric_vec_b)))
print("Cosine: {:.3f}".format(spatial.distance.cosine(numeric_vec_a, numeric_vec_b)))
print("Canberra: {:.3f}".format(spatial.distance.canberra(numeric_vec_a, numeric_vec_b)))
print("Cityblock: {:.3f}".format(spatial.distance.cityblock(numeric_vec_a, numeric_vec_b)))

boolean_vec_a, boolean_vec_b = np.random.randint(0, 2, size=(2, 20), dtype=bool)

print("\nDice: {:.3f}".format(spatial.distance.dice(boolean_vec_a, boolean_vec_b)))
print("Hamming: {:.3f}".format(spatial.distance.hamming(boolean_vec_a, boolean_vec_b)))
print("Jaccard: {:.3f}".format(spatial.distance.jaccard(boolean_vec_a, boolean_vec_b)))


A useful feature of `scipy.spatial` are the efficient batch distance computation functions. Say you want to know the pairwise cosine distances betweeen a set of 1000 300-dimensional vectors, which is a common thing when e.g. dealing with word vectors in NLP.

In [ ]:
vectors = np.random.randint(0, 100, size=(1000, 300))

You could do that with a `for`-loop and `scipy.spatial.distance.cosine`, which might be intuitive.

In [ ]:
import time 

start_time = time.time()

pairwise_distances_loop = np.zeros((1000, 1000))

for a_ix, vector_a in enumerate(vectors):
    for b_ix, vector_b in enumerate(vectors):
        pairwise_distances_loop[a_ix, b_ix] = spatial.distance.cosine(vector_a, vector_b)
        
print("This took {:.3f} seconds.".format(time.time() - start_time))

However, you will find that this tends to take a while.  

As an alternative, you could use the function [`scipy.spatial.distance.pdist`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html#scipy.spatial.distance.pdist), which is specialised for this task. 

In [ ]:
start_time = time.time()

pairwise_distances_nonsquare = spatial.distance.pdist(vectors, metric="cosine")
pairwise_distances_scipy = spatial.distance.squareform(pairwise_distances_nonsquare) # convert to square form

print("This took {:.3f} seconds.".format(time.time() - start_time))

It is orders of magnitude faster, and the result is pretty much the same except for small rounding errors due to the different implementations.

In [ ]:
print("Cumulative difference: {:.12f}".format((pairwise_distances_loop - pairwise_distances_scipy).sum()))

assert np.allclose(pairwise_distances_loop, pairwise_distances_scipy)

## Statistics and Distributions


`scipy` also has an elaborate statistics submodule, [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html). Here you can find your favourite discrete and continuous distributions, as well as some tests and coefficients.

In [ ]:
from scipy import stats

Let's take samples from a few distributions. To do that, we call [`scipy.stats.rv_discrete.rsv`](`https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_discrete.rvs.html#scipy.stats.rv_discrete.rvs) for discrete distributions and [`scipy.stats.rv_continuous.rsv`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_continuous.rvs.html#scipy.stats.rv_continuous.rvs) for continuous distributions.

In [ ]:
import seaborn as sns

n = 5000 # number of samples we will draw

samples = [
    ("Bernoulli Samples", stats.bernoulli.rvs(p=0.5, size=n)),
    ("Poisson Samples", stats.poisson.rvs(mu=0.5, size=n)),
    ("Geometric Samples", stats.geom.rvs(p=0.5, size=n)),
    ("Chi2 Samples", stats.chi2.rvs(df=4, size=n)),
    ("Normal Samples", stats.norm.rvs(size=n)),
    ("Uniform Samples", stats.uniform.rvs(size=n))
]

    
fig, axes = plt.subplots(nrows=2, ncols=3)

for ax, (dist_name, dist_samples) in zip(axes.flat, samples):
    sns.distplot(dist_samples, ax=ax).set_title(dist_name)
    
plt.tight_layout()
plt.show()

We can also compute correlation coefficients such as *Spearman's coefficient* with [`scipy.stats.spearmanr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html#scipy-stats-spearmanr) and *Pearson's coefficient* with [`scipy.stats.pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr).

In [ ]:
a = np.arange(10)
b = np.random.random(10)

print("a:", a)
print("b:", b)

coefficient, p_value = stats.spearmanr(a, b)
print("\nSpearman's coeffient is {:.3f} at a p-value of {:.3f}".format(coefficient, p_value))

coefficient, p_value = stats.pearsonr(a, b)
print("Pearson's coeffient is {:.3f} at a p-value of {:.3f}".format(coefficient, p_value))

## Homework

You can find the assignment link for `2021-homework10` in an announcement on StudIP. 

Additionally, a bonus homework assignment `2021-homework10-bonus` will be released on Wednesday. It does not count towards the `n` of the `n-2` rule, so you will not have any disadvantage if you do not hand it in. However, if you would like to catch up by one homework, this is a good opportunity.

>Good luck and have a good week!